In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
txt = "Data\Data 2.txt"
df = pd.read_csv(txt, sep="\t")

In [3]:
df.set_index("type", inplace = True)
df.drop(index = ["type"], inplace = True)
df.reset_index(inplace = True)
# df.head(20)

In [4]:
df.fillna(method = 'ffill', inplace = True)
df["eos"].fillna(1, inplace = True)
df["desc"].fillna(df['name'], inplace = True)
# df.head(10)

C:\Users\LUNA\AppData\Local\Temp\ipykernel_11220\2458029604.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method = 'ffill', inplace = True)
C:\Users\LUNA\AppData\Local\Temp\ipykernel_11220\2458029604.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['Area' 'Area' 'Area' ... 'Trees' 'Trees' 'Trees']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df["desc"].fillna(df['name'], inplace = True)


In [5]:
for col in df.columns:
    if df[col].nunique() <= 2:
        df.drop(columns = [col], inplace = True)

df.drop(columns = ['color', 'name'], inplace = True)
df.head(10)

,latitude,longitude,altitude (m),desc
0,-1.100108692,37.012773786,1521.2,Area
1,-1.099980347,37.013731104,1522.5,Area
2,-1.098893409,37.013679704,1522.7,Area
3,-1.098296925,37.013653718,1523.2,Area
4,-1.098277510,37.012825036,1522.2,Area
5,-1.098359120,37.012338111,1520.8,Area
6,-1.098575147,37.011742448,1520.1,Area
7,-1.100108692,37.012773786,1521.2,Area
8,-1.098860257,37.012802983,1521.6,Building
9,-1.098835477,37.013408132,1522.8,Building


In [6]:
gdf = gpd.GeoDataFrame(data = df, crs = "EPSG:4326", geometry = gpd.points_from_xy(df.longitude, df.latitude)) # type: ignore
gdf.drop(columns = ["longitude", "latitude"], inplace = True)

gdf['fid'] = gdf.index + 1
fid = gdf.pop('fid')
gdf.insert(0, 'fid', fid)
gdf.head(10)

,fid,altitude (m),desc,geometry
0,1,1521.2,Area,POINT (37.01277 -1.10011)
1,2,1522.5,Area,POINT (37.01373 -1.09998)
2,3,1522.7,Area,POINT (37.01368 -1.09889)
3,4,1523.2,Area,POINT (37.01365 -1.09830)
4,5,1522.2,Area,POINT (37.01283 -1.09828)
5,6,1520.8,Area,POINT (37.01234 -1.09836)
6,7,1520.1,Area,POINT (37.01174 -1.09858)
7,8,1521.2,Area,POINT (37.01277 -1.10011)
8,9,1521.6,Building,POINT (37.01280 -1.09886)
9,10,1522.8,Building,POINT (37.01341 -1.09884)


In [50]:
a = -1382256.37087345
b = 1.00479613446328
c = 0.139791893088955
d = 1511719.78927612
e = -0.0218098151381128
f = 0.847560128597252

In [51]:
# Transform to corresponding coordinates
gdf2 = gpd.GeoDataFrame((gdf.affine_transform([a, b, c, d, e, f])))
gdf2.head(10)


,0
0,POINT (-51161143.49659 -1663050.05841)
1,POINT (-51162466.75537 -1662856.03660)
2,POINT (-51162395.70630 -1661212.89092)
3,POINT (-51162359.78638 -1660311.17425)
4,POINT (-51161214.33539 -1660281.82433)
5,POINT (-51160541.28029 -1660405.19585)
6,POINT (-51159717.92153 -1660731.76823)
7,POINT (-51161143.49659 -1663050.05841)
8,POINT (-51161183.85308 -1661162.77451)
9,POINT (-51162020.32411 -1661125.31401)
